# Пример решения

In [1]:
import pandas
import xarray
import requests
import datetime
import numpy as np
from tqdm import tqdm_notebook as tqdm

## NCEP Dataset

Погодные данные из проекта [NCEP Reanalysis 2](https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis2.html) — усреднённые за день температура воздуха, относительная влажность и компоненты ветра. Данные можно получить с 1979 года.

Загрузите наборы данных в каталог `data/ncep/`:
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/air.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/uwnd.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/rhum.2018.nc

In [4]:
ncep_data = []
year = 2018
#uwnd -   u = ws * cos(θ) http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv

for var in ('air', 'uwnd', 'rhum'):
    dataset_filename = 'data/ncep/{}.{}.nc'.format(var, year)
    ncep_data.append(xarray.open_dataset(dataset_filename))
ncep_data = xarray.merge(ncep_data)

/home/nikolay/.pyenv/versions/3.7.3/envs/main/lib/python3.7/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'air' has multiple fill values {-99, 32767}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
/home/nikolay/.pyenv/versions/3.7.3/envs/main/lib/python3.7/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'uwnd' has multiple fill values {-99, 32767}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
/home/nikolay/.pyenv/versions/3.7.3/envs/main/lib/python3.7/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'rhum' has multiple fill values {-99, 32767}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


In [17]:
ncep_data

<xarray.Dataset>
Dimensions:  (lat: 73, level: 12, lon: 144, time: 365)
Coordinates:
  * level    (level) float64 50.0 70.0 100.0 150.0 ... 500.0 700.0 850.0 1e+03
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-12-31
Data variables:
    air      (time, level, lat, lon) float32 ...
    uwnd     (time, level, lat, lon) float32 ...
    rhum     (time, level, lat, lon) float32 nan nan nan ... 88.31999 88.31999

## Набор признаков на основе данных NCEP

Ищем наиболее близкий к точке узел сетки в наборе NCEP, в качестве признаков значения переменных зарегистрированные в день регистрации точки и агрегированные показатели за период от 1 до 3х недель до момента регистрации точки.

In [5]:
def extract_features(row):
    point = ncep_data.sel(
        lon=row['longitude'],
        lat=row['latitude'],
        level=1000,
        method='nearest',
    )

    p1w = point.rolling(time=7).mean()
    p2w = point.rolling(time=14).mean()
    p3w = point.rolling(time=21).mean()
    
    date = row['date']
    v = point.sel(time=date)
    v1w = p1w.sel(time=date)
    v2w = p2w.sel(time=date)
    v3w = p3w.sel(time=date)
    
    return {
        'fire_id': row['fire_id'],
        'fire_type': row['fire_type'],
        'fire_type_name': row['fire_type_name'],
        'date': row['date'], 
        'temperature': v.air.values.item(0),
        'humidity': v.rhum.values.item(0),
        'uwind': v.uwnd.values.item(0),
        't1w': v1w.air.values.item(0),
        't2w': v2w.air.values.item(0),
        't3w': v3w.air.values.item(0),
        'h1w': v1w.rhum.values.item(0),
        'h2w': v2w.rhum.values.item(0),
        'h3w': v3w.rhum.values.item(0)
    }

## Выборка для обучения

In [51]:
df_train = pandas.read_csv('data/wildfires_train.csv')
df_subsample = df_train.query('(date > "2018") & (date < "2019")').sample(n=2000)

df_features = []
for i, row in tqdm(df_subsample.iterrows(), total=df_subsample.shape[0]):
    features = extract_features(row)
    df_features.append(features)
df_features = pandas.DataFrame(df_features)
df_features.set_index('fire_id', inplace=True)

/home/nikolay/.pyenv/versions/3.7.3/envs/main/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [25]:
df_train

,fire_id,date,latitude,longitude,fire_type,fire_type_name
0,0,2012-01-01,42.913439,133.887370,4,сжигание порубочных остатков
1,1,2012-01-01,43.378618,131.772265,3,горение мусора
2,2,2012-01-01,42.634134,130.479116,4,сжигание порубочных остатков
3,3,2012-01-02,43.108370,132.001058,11,не подтверждено
4,4,2012-01-02,42.890825,131.337416,4,сжигание порубочных остатков
...,...,...,...,...,...,...
174866,174866,2019-04-30,57.403435,40.877734,9,природный пожар
174867,174867,2019-04-30,54.739667,32.782476,11,не подтверждено
174868,174868,2019-04-30,53.874580,55.240782,9,природный пожар
174869,174869,2019-04-30,56.134949,32.781693,9,природный пожар


In [101]:
names = df_train['fire_type_name'].value_counts().index.tolist()
codes = df_train['fire_type'].value_counts().index.tolist()
decode_prediction_dict = dict(zip(names, codes))
decode_prediction_dict

{'неконтролируемый пал': 6,
 'не подтверждено': 11,
 'природный пожар': 9,
 'контролируемый пал': 10,
 'лесной пожар': 8,
 'сжигание порубочных остатков': 4,
 'горение мусора': 3,
 'сжигание мусора': 5,
 'технологический процесс': 1,
 'техногенный пожар': 2,
 'торфяной пожар': 7}

In [46]:
import requests
from datetime import datetime


url = "https://community-open-weather-map.p.rapidapi.com/onecall/timemachine"
headers = {
    'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
    'x-rapidapi-key': "5c9fe5fbf4msh235ad87d04a08ebp1c5a36jsnc0d452f37968"
    }


for i in df_subsample.index[:1]:
    lat, long, date = df_train.loc[i, ['latitude', 'longitude', 'date']]
    
    querystring = {"lat": f"{lat}", "lon": f"{long}", "dt": f"{datetime(2012,4,1,0,0).timestamp()}"}

    response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message":"You are not subscribed to this API."}


In [102]:
API_key = 'af70d38caa534b668df5530807544926'
start = datetime(2012,4,1,0,0).timestamp()

#f"http://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={long}&type=hour&start={start}&appid={API_key}"


url = f"http://history.openweathermap.org/data/2.5/history/city"

#f"lat={lat}&lon={long}&type=hour&start={start}&appid={API_key}"

querystring = {"lat": f"{lat}", "lon": f"{long}", "start": f"{start}", "appid": f"{API_key}", "type": "hour"}

response = requests.request("GET", url, params=querystring)
response.json()

{'cod': 401,
 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}

## Обучение классификатора

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [76]:
X = df_subsample.iloc[:, 2:4].fillna(0)
y = df_subsample['fire_type']
X

,latitude,longitude
138204,43.526663,135.117429
137272,50.727760,106.729544
146549,53.857441,110.024047
137718,52.594726,126.570702
156475,48.909194,132.767417
...,...,...
138537,45.203406,133.558313
149211,52.562177,43.704252
139249,46.327097,47.467015
142403,51.485294,130.405738


In [77]:
fire_classifier = GradientBoostingClassifier()

In [78]:
cross_val_score(
    fire_classifier, 
    X, y, 
    cv=5, 
)

array([0.48  , 0.4825, 0.485 , 0.48  , 0.5225])

In [79]:
fire_classifier.fit(X, y)

GradientBoostingClassifier()

## Решение для отправки в систему

In [80]:
import pickle

with open('solution/model.pickle', 'wb') as fout:
    pickle.dump(fire_classifier, fout, protocol=pickle.HIGHEST_PROTOCOL)

## Прогнозирование на новых данных

In [81]:
df_predictions = pandas.DataFrame(
    fire_classifier.predict_proba(X),
    index=df_features.index,
    columns=[
        'fire_{}_prob'.format(class_id)
        for class_id in fire_classifier.classes_
    ],
)

df_predictions.head()

,fire_1_prob,fire_2_prob,fire_3_prob,fire_4_prob,fire_5_prob,fire_6_prob,fire_8_prob,fire_9_prob,fire_10_prob,fire_11_prob
fire_id,,,,,,,,,,
138204,0.000192,0.000002,0.000897,0.002294,0.000319,0.826256,0.045188,0.029259,0.074148,0.021444
137272,0.005280,0.000006,0.046236,0.091316,0.016611,0.155824,0.090024,0.067190,0.510424,0.017089
146549,0.001575,0.000004,0.021139,0.336363,0.020977,0.119337,0.238901,0.023204,0.229510,0.008990
137718,0.000706,0.000003,0.016069,0.044195,0.004027,0.045681,0.149190,0.172400,0.554666,0.013062
156475,0.000625,0.000008,0.012751,0.005974,0.001036,0.356908,0.096589,0.338528,0.163047,0.024533


In [16]:
df_predictions.to_csv('data/sample_predictions.csv', index_label='fire_id')

In [91]:
model = pickle.load(open('solution/model.pickle', 'rb'))

pred_data = np.array([43.526663, 135.117429])
prediction = model.predict([pred_data])


array([6])